# Product-based Recommendation System
## Based on Moorissa Tjokro tutorial
### Coded by Rebeca Bivar - DB: Armazem Paraíba

### Imports and reading file (data here has only clients, products and purchase count)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import turicreate as tc
from sklearn.model_selection import train_test_split


In [2]:
#READING A FILE WITH ONLY CLIENTS, PRODUCTS BOUGHT AND QTDE
buyers = pd.read_csv('teste.csv', sep=';')
buyers.head()

,COD_CLIENTE,COD_PRODUTO,QUANTIDADE
0,00000301,25243,1
1,00001501,25726,1
2,00002001,21653,1
3,00002701,25303,1
4,00003101,21255,1


##  Data preparation
### Creating dummy table to check if the client has bought a product or not

In [3]:
def create_data_dummy(db):
    data_dummy = db.copy()
    data_dummy['purchase_dummy'] = '1'
    return data_dummy

data_dummy = create_data_dummy(buyers)
data_dummy


,COD_CLIENTE,COD_PRODUTO,QUANTIDADE,purchase_dummy
0,00000301,25243,1,1
1,00001501,25726,1,1
2,00002001,21653,1,1
3,00002701,25303,1,1
4,00003101,21255,1,1
...,...,...,...,...
311736,YZ399601,25996,1,1
311737,YZ399801,26327,1,1
311738,YZ400601,26432,1,1
311739,YZ401401,26432,1,1


### Normalizing purchase frequency of each item across users 

In [4]:
#Dummy for marking whether a customer bought that item or not

df_matrix = pd.pivot_table(buyers, values = 'QUANTIDADE', index = 'COD_CLIENTE', columns = 'COD_PRODUTO')

df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
#df_matrix_norm

# create a table for input to the modeling  
data_input = df_matrix_norm.reset_index()
data_input.index.names = ['FREQ_COMPRAS']
data_norm = pd.melt(data_input, id_vars=['COD_CLIENTE'], 
                    value_name='FREQ_COMPRAS')

#print(data_norm.shape)
#data_norm.head()

In [9]:
#Just cleaning useless values 
data_norm = data_norm.dropna()
data_norm.head()

,COD_CLIENTE,COD_PRODUTO,FREQ_COMPRAS
1181239,00017101,11332,0.25
1184028,00449301,11332,1.00
1184343,00846001,11332,0.00
1185377,02130901,11332,0.00
1186365,03152901,11332,0.00


### Split data into trainning and testing (80/20)

In [12]:

# Returns train and test datasets as scalable dfs
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

# Now actually splitting purchase_counts, purchase_dummy and purchase_counts_norm
train_data, test_data = split_data(buyers)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## Baseline model to compare and evaluate models
### Popularity Model
   Takes the most popular items for recommendation, which are the products with the highest number of sells across customers.


In [25]:
# variables to define field names: 

user_id = 'COD_CLIENTE'
item_id = 'COD_PRODUTO'
users_to_recommend = list(buyers[user_id])
n_recommendation = 10 # itens to recommend
n_display = 30 # display the first few rows in an output dataset

# Function for all models using turicreate
def model(train_data, name, user_id, item_id, target, 
          users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id = user_id, 
                                                    item_id = item_id,
                                                    target = target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

### Using purchase count - Buyers

In [27]:
name = 'popularity'
target = 'QUANTIDADE'
popularity = model(train_data, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)
popularity

Preparing data set.

Data has 249392 observations with 198072 users and 670 items.

Data prepared in: 0.183193s

249392 observations to process; with 670 unique items.

recommendations finished on 1000/311741 queries. users per second: 132083

...

recommendations finished on 307000/311741 queries. users per second: 548180

recommendations finished on 308000/311741 queries. users per second: 544723

recommendations finished on 309000/311741 queries. users per second: 541862

recommendations finished on 310000/311741 queries. users per second: 538838

recommendations finished on 311000/311741 queries. users per second: 535544

### Using scaled purchase count 

In [29]:
name = 'popularity'
target = 'FREQ_COMPRAS'
pop_norm = model(train_data_norm, name, user_id, item_id, target, 
                 users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 245474 observations with 195699 users and 308 items.

Data prepared in: 0.167097s

245474 observations to process; with 308 unique items.

recommendations finished on 1000/311741 queries. users per second: 109830

...

recommendations finished on 307000/311741 queries. users per second: 728918

recommendations finished on 308000/311741 queries. users per second: 727921

recommendations finished on 309000/311741 queries. users per second: 726522

recommendations finished on 310000/311741 queries. users per second: 723619

recommendations finished on 311000/311741 queries. users per second: 720087

### Collaborative Filtering Model

   Recommends items based on how similar clients purchase items. Meaning: if customer 1 and customer 2 bought similar items, for example, 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.
    
   - Lets say X and Y have been rated by costumers 1 and 2. 
   - We then create two item-vectors for both items, then we find the **cosine** or **pearson** distance between these vectors. If the **cosine** value is 1, means total similarity, if it is 0, means no similarity.
   - In this case, we will check the similarity between the target item and other items the customer already bought - using the client's purchase count to items already bought by him as weighing factor (some sort of simulated rating). 

### Using purchase count - Buyers

In [30]:
name = 'cosine'
target = 'QUANTIDADE'
cos = model(train_data, name, user_id, item_id, target, 
            users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 249392 observations with 198072 users and 670 items.

Data prepared in: 0.169499s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 307000/311741 queries. users per second: 405427

recommendations finished on 308000/311741 queries. users per second: 403979

recommendations finished on 309000/311741 queries. users per second: 402103

recommendations finished on 310000/311741 queries. users per second: 400063

recommendations finished on 311000/311741 queries. users per second: 398162